In [0]:
# Colab 相关设置项
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes


In [0]:
%%capture
%run "第 12 章：二叉搜索树.ipynb"  # 导入 BinarySearchTree

In [0]:
import collections
import random

## 13.1 红黑树的性质

#### 红黑树的定义及性质

- 红黑树是许多平衡搜索树中的一种，可以保证在最坏的情况下，基本动态操作的时间复杂度为 $O(\lg n)$
  - 红黑树是一棵二叉搜索树，其在每个结点上增加一个存储位来表示结点的颜色。其确保没有一条路径会比期它路径长出 2 倍，因而近似
  是平衡的
  - 红黑树的每个结点包括 5 个属性： $color, key, left, right$ 和 $p$，如果一个结点没有子结点或父结点，则该结点相应的
  指针属性值为 $\mathrm{NIL}$
  - 这些 $\mathrm{NIL}$ 可被看做指向二叉搜索树叶结点的指针，即视为**外部结点**，而将带有关键字的结点视为树的**内部结点**
  - 红黑树是满足下面 $5$ 条**红黑性质**的二叉搜索树
    1. 每个结点或是红色或是黑色
    2. 根结点是黑色的
    3. 每个叶结点(外部结点) 是黑色的
    4. 如果一个结点是红色的，则它的两个子结点都是黑色的
      - 即一个红色结点的父结点是黑色的
      - 即一条简单的路径上，不可能同时出现 2 个红色结点
    5. 对每个结点，从该结点至其所有后代叶结点的简单路径上，均包含相同数目的黑色结点，由此可定义黑高的概念
      - 黑高(black height) 的定义
        - 从某个结点 $x$ 出发（不含该结点）到达一个叶结点的任意一条简单路径上的黑色结点个数称为该结点的黑高，记为 
        $bh(x)$

#### 红黑树的示意图

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216145017.png width=800>

##### 哨兵 $T.nil$

- 使用哨兵结点 $T.nil$ 来替代 $\mathrm{NIL}$，，即可以用其来代表所有的叶结点和根结点的父结点，哨兵除了颜色需要为黑色外，
其余的属性并不重要，但在程序中为了方便，可以将它们设置为特定的值
- 示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216145122.png width=750>

##### 实际表示时，也可以省略掉叶结点

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216145209.png width=800>

#### 引理 13.1
- 一棵有 $n$ 个内部结点的红黑树的高度至多为 $2lg(n+1)$

- 证明：
  - 先证明以任一结点 $x$ 为根的子树中至少包含 $2^{bh(x)}-1$ 个内部结点
    - 可采用数归纳法证明，当 $bh(x)==0$ 时，则 $x=T.nil$,树高为 $0$ ，内部结点为 0，符合要求
    - 归纳步骤
      - 对于黑高为$bh(x)$ 的结点，其左右子结点的黑高为 $bh(x)$ 或 $bh(x)-1$，取决于$x$ 结点的颜色，由归纳法可得以 
      $x$ 为根的子树，至少包含 $(2^{bh(x)-1} - 1) + (2^{bh(x)-1} - 1) + 1 = 2^{bh(x)} - 1$ 个内部结点
  - 设树高为 $h$，则由红黑树的性质4可知，从根结点到叶结点，黑高至少为 $h/2$，可得：
    - $bh(x) \ge \frac{h}{2}$， 即 $n \ge 2^{h/2} - 1$
    - 由此可得 $h \leq 2lg(n+1)$

- 由此引理可知，红黑树上的动态集合操作 $SEARCH, MINIMUM, MAXIMUM, SUCCESSOR$ 和 $PREDECESSOR$ 可以在 $O(\lg n)$ 的时间
内执行完成


#### 代码实现

###### 红黑树结点的代码实现

In [0]:
class RBNode():
  R = False
  B = True

  def __init__(self, left=None, right=None, parent=None, color=R, key=None):
    self.left = left
    self.right = right
    self.parent = parent
    self.color = color
    self.key = key
  
  def __repr__(self):
    key = self.key if self.key is not None else 'None'
    color = 31 if self.color == RBNode.R else 37  # 为了对比明显，用白色表示黑色
    return '\033[0;{}m{}\033[0m'.format(color, key)
  
  def __eq__(self, other):
    return self.key == None if other is None else self is other  # 为了兼容二叉搜索树中的相关函数

In [0]:
a = RBNode(key = 66)
a

66

In [0]:
b = RBNode(key = 66, color=RBNode.B)
b

66

###### 红黑树代码的部分实现

In [0]:
class RBTree(BinarySearchTree):
  NIL = RBNode(color=RBNode.B) # 黑色的 T.Nil 结点

  def __init__(self, root=NIL):
    self.root = root

  def _binary_tree_insert(self, z):
    """普通二叉树插入，不考虑结点的颜色是否符合要求"""
    super().insert(z)
    z.left = RBTree.NIL
    z.right = RBTree.NIL
    if self.root == z:
      z.parent = RBTree.NIL


In [0]:
rbt = RBTree()
all_nodes = [26, 17, 41, 14, 21, 30, 47, 10, 16, 19, 23, 28, 38, 7, 12, 15, 20, 35, 39, 3]
black_nodes = [26, 41, 14, 21, 47, 16, 19, 23, 28, 38, 7, 12]
for key in all_nodes:
  rbt._binary_tree_insert(RBNode(key=key))
for key in black_nodes:
  node = rbt.search(rbt.root, key)
  node.color = RBNode.B
rbt


|  |  47

|  41

|  |  |  |  39

|  |  |  38

|  |  |  |  35

|  |  30

|  |  |  28

26

|  |  |  23

|  |  21

|  |  |  |  20

|  |  |  19

|  17

|  |  |  16

|  |  |  |  15

|  |  14

|  |  |  |  12

|  |  |  10

|  |  |  |  7

|  |  |  |  |  3

### 练习

#### 13.1-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221194057.png width=800>

- 先取关键字中的中位数，作为根结点，并以根结点将数据划分为左右数组。然后分别取左右数组中的中位数作为左右结点，以此递推，即可得到如下图的结果
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221193931.png width=400>
- 黑高为 2 的红黑树
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221195027.png width=600>
- 黑高为 3 的红黑树
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221195546.png width=600>
- 黑高为 4 的红黑树
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221195635.png width=600>


#### 13.1-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221200015.png width=800>

- 如果结点为红色，则不满足性质 4
- 如果结点为黑色，则不满足性质 5

###### 13.1-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221200755.png width=800>

- 根结点变为黑色即满足性质2，而其不会改变其它条的性质，因此所得的仍是一棵红黑树

#### 13.1-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221201102.png width=800>

- 一个结点的度等于二叉树中以该结点为端点的边数
- 将红结点“吸叫”到其黑色的父结点中，所得的即为 2-3-4 树，即一个结点可能含有2个，3个或4个子结点
  - 如果一个黑色结点只有 1 个红色子结点，则吸叫后即有 3 个子结点
  - 如果一个黑色结点有两个红色的子结点，则吸叫后即有 3 个子结点
- 因此可能的度为 0 至 5
- 红色结点被全部吸叫，则树的叶线点的深度至多减小至原来的 $1/2$

#### 13.1-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221202321.png width=800>

- 由性质 5 可知，最短的一条一定为全为黑色的路径，最长的一条为包含红色结点最多的一条
- 由性质 4 可知，从结点 $x$ 至其后代叶结点的所有简单路径中，内部的红色结点至多与黑色结点一样多
- 设最短路径长度为 $t$，则其黑高为 $t$
- 设最长路径为 $s$，则 $s$ 中有 $t$ 个内部的黑色结点，则至多有 $t$ 个内部的红色结点，外加一个
黑色的叶子结点，则可得 $s \le (t + t + 1) - 1 = 2t$ 

#### 13.1-6
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221204722.png width=800>

- 如果为全黑，则内部结点最少，为 $2^{k+1} - 1$
- 由 13.1-5 可知，从根结点到叶结点的一条简单路径上，其内部的红色结点至多和黑色结点一样多，则树高至多为 $2k$，
即内部结点最多为 $2^{2k+1} - 1$

#### 13.1-7
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221205614.png width=800>


- 比值最大为 2，即下黑色的结点有两个红色的子结点，其余全部为叶子结点
- 比值最小为 0，即全为黑色结点

## 13.2 旋转

- 红黑树的插入和删除操作，需要修改指针结构才能保证红黑树的性质。而为了保持二叉搜索树的性质，指针的修改需要通过**旋转**来完成

- 左旋与右旋
  - 示意图
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216152802.png width=650>
  - 左旋需要假设结点的右子结点不为 $T.nil$，右旋需要假设结点的左子结点不为 $T.nil$

#### 代码实现

##### 左旋操作

In [0]:
def left_rotate(self, x):
  """左旋操作，需要确保 $x.right 存在$"""
  y = x.right
  x.right = y.left
  if y.left != RBTree.NIL:
    y.left.parent = x
  y.left = x
  y.parent = x.parent
  y.left = x
  
  if x.parent == RBTree.NIL:
    self.root = y
  elif x.parent.left == x:
    x.parent.left = y
  else:
    x.parent.right = y

  x.parent = y


RBTree.left_rotate = left_rotate

In [0]:
rbt = RBTree()
black_nodes = [7, 4, 11, 3, 6, 9, 18, 2, 14, 19, 12, 17, 22, 20]
for key in black_nodes:
  rbt._binary_tree_insert(RBNode(key=key, color=RBNode.B))
print("原始的红黑树为：\n{}".format(rbt))

i = 11
rbt.left_rotate(rbt.search(rbt.root, i))
print("右旋结点{}后，红黑树为：\n{}".format(i, rbt))


原始的红黑树为：

|  |  |  |  22

|  |  |  |  |  20

|  |  |  19

|  |  18

|  |  |  |  17

|  |  |  14

|  |  |  |  12

|  11

|  |  9

7

|  |  6

|  4

|  |  3

|  |  |  2

右旋结点11后，红黑树为：

|  |  |  22

|  |  |  |  20

|  |  19

|  18

|  |  |  |  17

|  |  |  14

|  |  |  |  12

|  |  11

|  |  |  9

7

|  |  6

|  4

|  |  3

|  |  |  2



###### 示意图

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216154956.png width=600>

##### 右旋操作

In [0]:
def right_rotate(self, x):
  """右旋操作，前提是 $x.left$ 存在"""
  y = x.left
  x.left = y.right
  if y.right != RBTree.NIL:
    y.right.parent = x
  
  y.parent = x.parent
  y.right = x

  if x.parent == RBTree.NIL:
    self.root = y
  elif x.parent.left == x:
    x.parent.left = y
  else:
    x.parent.right = y

  x.parent = y

RBTree.right_rotate = right_rotate

In [0]:
print("原始的红黑树为：\n{}".format(rbt))

i = 18
rbt.right_rotate(rbt.search(rbt.root, i))
print("右旋结点{}后，红黑树为：\n{}".format(i, rbt))

原始的红黑树为：

|  |  |  22

|  |  |  |  20

|  |  19

|  18

|  |  |  |  17

|  |  |  14

|  |  |  |  12

|  |  11

|  |  |  9

7

|  |  6

|  4

|  |  3

|  |  |  2

右旋结点18后，红黑树为：

|  |  |  |  22

|  |  |  |  |  20

|  |  |  19

|  |  18

|  |  |  |  17

|  |  |  14

|  |  |  |  12

|  11

|  |  9

7

|  |  6

|  4

|  |  3

|  |  |  2



### 练习

### 13.2-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221210048.png width=750>

- 证明：
  - 采用归纳法，如果 $n = 1$，则只有 $0$ 种旋转的可能，结论成立
  - 归纳步骤
    - 如果有 $n$ 个结点的二叉搜索树没有左子结点或右子结点，则其子树的结点个数为 $n-1$，可能的旋转组合为
      - $$(n-1-1) + 1 = n -1$$
    - 如果其根结点既有左子树，又有右子树，设左子树的结点为 $k$，则右子树的结点数为 $n-k-1$，由归纳假设可得
    总的旋转可能为：
      - $$(k-1) + (n-k-1 - 1) + 2 = n - 1$$
  - 证毕

#### 13.2-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200221215352.png width=800>

- $a$ 增加 $1$， $b$ 不变， $c$ 减少 $1$

#### 13.2-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/%E6%97%A0%E6%A0%87%E9%A2%98.png width=800>

- 设一棵二叉搜索树中右侧伸展的链上的结点组合为集合 S ，每对 $S$ 上的元素进行一次右旋操作，则 $S$ 中的元素
会增加 $1$，而 $S$ 中已有的元素却不会改变。 起始时， $S$ 中的元素数量至少为 $1$，所以至多 $n-1$ 次右旋即
可将树转变为一条右侧伸展的链
- 设有两题树 $T_1, T_2$，设 $T_1$ 经过 $x_1, x_2, \cdots,x_{m_1}$ 次右旋变为右侧伸展的链，
$T_2$ 经过 $y_1, y_2, \cdots,y_{m_2}$ 次右旋变为右侧伸展的链，两者转变而成的右侧伸展的链是相同的，因此 $T_1$
经过旋转操作 $x_1, x_2, \cdots, x_{m_1}, y_{m_2}^{'}, y_{m_2 - 1}^{'}, \cdots, y_2, y_1$ 即可变为
$T_2$，其中 $y_m^{'}$ 为 $y_m$ 的反向操作

#### 13.2-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224085627.png width=800>

- $T_1$ 不能转换为 $T_1$ 的例子
  - $T_1$ 树为
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224085948.png width=500>
  - $T_2$ 树为
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224181903.png width=500>
- 假设 $T_1$ 能够通过右旋变为 $T_2$，则 $T_2$ 的根结点一定在 $T_1$ 的左脊柱中(left spline)
  - 只通过右旋，右子树的元素不可能进入左子树中
- $T_1$ 左脊柱的长度至多为 $O(n)$，然后变换需要 $O(n)$ 次右旋。分别对 $T_2$ 的左右子树进行同样的操作，
可得： $$T(n) = T(k) + T(n-k-1) + O(n)$$
  - 假设 $T_2$ 左子树中有 $k$ 个元素
  - 如此用代入法进入证明，即可得 $T(n) = O(n^2)$




## 13.3 插入

- 假设要插入的结点为 $z$，可以先将 $z$ 按照满二叉搜索树性质的方法将 $z$ 插入二叉树 $T$ 中，并将 $z.colort$ 设置为红色，
然后调用 $\mathrm{RB-INSERT-FIXUP}$ 来保持红黑树的性质

#### $\mathrm{RB-INSERT-FIXUP}$

- 如果 $z$ 为根结点，需要将其颜色设为黑色
- 其它情况下，只有当 $z.parent$ 为红色时，才会违反性质，共分为三种情况
  - $z.parent$ 是 $z.parent.paent$ 的左孩子还是右孩子的情况基本对称，后续分析时假设其为左孩子

##### 情况1： $z$ 的叔结点 $y$ 是红色

- 此时更改相应结点的颜色后将 $z$ 上移即可
- 示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216161343.png width=650>

##### 情况2: $z$ 的叔结点是黑色且 $z$ 是一个右孩子

- 此种情况，通过旋转操作，即可将 $z$ 变为左孩子，情况 2 即可转变为情况 3
- 旋转的目的是为了使 $z, z.parent$ 与 $z.parent.parent$ 在一条直线上

##### 情况3： $z$ 的叔结点是红色且 $z$ 是一个左孩子

- 通过旋转操作，即可使得 $T$ 满足红黑树的所有性质要求
- 情况 2 和情况 3 的示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216162316.png width=650>

#### 分析

- 时间复杂度为 $O(\lg n)$，因为 $\mathrm{RB-INSERT-FIXUP}$ 会沿着树上升，最高到根结点，而且所做的旋转操作最多只有 2 次，因此时间复杂度为
$O(h)$， 即 $O(\lg n)$

#### 代码实现

In [0]:
def insert(self, z):
  """红黑树的插入操作，直接调用二叉树的插入操作即可，不过要将相就的 None 替换为 nil"""
  super(RBTree, self).insert(z)
  if self.root == z:
    z.parent = RBTree.NIL
  z.left = RBTree.NIL
  z.right = RBTree.NIL
  z.color = RBNode.R

  self.insert_fixup(z)

RBTree.insert = insert

In [0]:
def insert_fixup(self, z):
  while z.parent.color == RBNode.R:
    if z.parent == z.parent.parent.left:
      y = z.parent.parent.right
      if y.color == RBNode.R:  # case 1
        z.parent.color = RBNode.B
        y.color = RBNode.B
        z.parent.parent.color = RBNode.R
        z = z.parent.parent
      else:
        if z.parent.right == z: # case 2
          z = z.parent
          self.left_rotate(z)
        z.parent.color = RBNode.B
        z.parent.parent.color = RBNode.R
        self.right_rotate(z.parent.parent)
    else:
      y = z.parent.parent.left
      if y.color == RBNode.R:  # case 1
        z.parent.color = RBNode.B
        y.color = RBNode.B
        z.parent.parent.color = RBNode.R
        z = z.parent.parent
      else:
        if z.parent.left == z: # case 2
          z = z.parent
          self.right_rotate(z)
        z.parent.color = RBNode.B
        z.parent.parent.color = RBNode.R
        self.left_rotate(z.parent.parent)
      

  self.root.color = RBNode.B

RBTree.insert_fixup = insert_fixup

In [0]:
rbt = RBTree()
all_nodes = [11, 2, 14, 1, 7, 15, 5, 8]
black_nodes = [11, 14, 1, 7]
for key in all_nodes:
  rbt._binary_tree_insert(RBNode(key=key))
for key in black_nodes:
  node = rbt.search(rbt.root, key)
  node.color = RBNode.B
print("原始红黑树为：\n{}".format(rbt))
i = 4
rbt.insert(RBNode(key=i))
print("插入结点 {} 后， 红黑树为：\n{}".format(i, rbt))

原始红黑树为：

|  |  15

|  14

11

|  |  |  8

|  |  7

|  |  |  5

|  2

|  |  1

插入结点 4 后， 红黑树为：

|  |  |  15

|  |  14

|  11

|  |  8

7

|  |  5

|  |  |  4

|  2

|  |  1



### 练习

#### 13.3-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224202529.png width=800>

- 插入黑色的结点虽可以保证性质4 不被破坏，但是一旦红黑树不为空，插入黑色结点，就会破坏性质 5
  - 从根结点到插入结点的叶子结点的简单路径中的黑色结点数要比到其它叶子结点的多 $1$

#### 13.3-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224202903.png width=800>

- 插入 41
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224203202.png width=50>
- 插入 38
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224203251.png width=200>
- 插入 31
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224203651.png width=400>
- 插入 12
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224203858.png width=500>
- 插入 19
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224204445.png width=600>
- 插入 8
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224204401.png width=600>


#### 13.3-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224205636.png width=800>

- 图 13-5 只是将 C 的黑色下移到其子结点中，不会影响各个子树的黑高
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224210020.png width=800>
- 图 13-6 进行旋转操作时，会互换 B, C 的颜色，也不会影响各个子树的黑高
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224210302.png width=800>

#### 13.3-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224210600.png width=800>

- 如果 $z.parent.color == red$，由性质 2 结合性质 4 可知 $z.p.p$ 一定存在，且不为 $NIL$， 则后续对
 $z.p.p$ 颜色的修改便不会影响到 $T.nil.color$

#### 13.3-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224211118.png width=800>

- 证明
  1. case1, 2, 3 中都会至少保证存在一个红色结点
  2. 如果插入的结点的父结点为黑色结点，则插入的结点即为红色，不需要修改
  - 由 1,2 可得如果 $n>1$，则树中至少有一个红色结点

#### 13.3-6
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200224211637.png width=800>

- 如果不提供父指针，则需要用一个栈来记录找到插入位置的父结点

##### 代码实现

###### 没有父结点的红黑树结点代码

In [0]:
class RBNodeWithoutParent(RBNode):
  def __init__(self, left=None, right=None, color=RBNode.R, key=None):
    self.left = left
    self.right = right
    self.color = color
    self.key = key

In [0]:
RBNodeWithoutParent(key=12)

12

###### 没有父指针的 RB-INSERT 代码

In [0]:
class RBTreeWithoutParent(RBTree):
  """没有父指针的红黑树"""
  def __init__(self, root=RBTree.NIL):
    super().__init__(root)

  def insert(self, z):
    parents = collections.deque()
    parents.append(RBTree.NIL)
    x = self.root
    y = RBTree.NIL
    while x != RBTree.NIL:
      y = x
      parents.append(x)
      if z.key < x.key:
        x = x.left
      else:
        x = x.right
    
    z.parent = y
    if y == RBTree.NIL:
      self.root = z
    elif z.key < y.key:
      y.left = z
    else:
      y.right = z
    z.left = RBTree.NIL
    z.right = RBTree.NIL
    z.color = RBNode.R
    self.insert_fixup(z, parents)
  
  def insert_fixup(self, z, parents):
    p = parents.pop()
    while p.color == RBNode.R:
      pp = parents.pop()
      if pp.left == p:
        if pp.right.color == RBNode.R:
          pp.left.color = RBNode.B
          pp.right.color = RBNode.B
          pp.color = RBNode.R
          z = pp
          p = parents.pop()
        else:
          if p.right == z:
            self.left_rotate(p, pp)
            z, p = p, z
          ppp = parents.pop()
          p.color = RBNode.B
          pp.color = RBNode.R
          self.right_rotate(pp, ppp)
      else:
        if pp.left.color == RBNode.R:
          pp.left.color = RBNode.B
          pp.right.color = RBNode.B
          pp.color = RBNode.R
          z = pp
          p = parents.pop()
        else:
          if p.left == z:
            self.right_rotate(p, pp)
            z, p = p, z
          ppp = parents.pop()
          p.color = RBNode.B
          pp.color = RBNode.R
          self.left_rotate(pp, ppp)
    self.root.color = RBNode.B

  def left_rotate(self, x, xparent):
    """左旋操作"""
    y = x.right
    x.right = y.left
    y.left = x
    if xparent == RBTree.NIL:
      self.root = y
    elif xparent.left == x:
      xparent.left = y
    else:
      xparent.right = y

  def right_rotate(self, x, xparent):
    """右旋操作"""
    y = x.left
    x.left = y.right
    y.right = x
    if xparent == RBTree.NIL:
      self.root = y
    elif xparent.left == x:
      xparent.left = y
    else:
      xparent.right = y



  

In [0]:
rbt = RBTreeWithoutParent()
for key in [41, 38, 31, 12, 19, 8]:
  rbt.insert(RBNode(key=key))
  print("插入 {} 后，红黑树为：\n{}".format(key, rbt))

插入 41 后，红黑树为：

41

插入 38 后，红黑树为：

41

|  38

插入 31 后，红黑树为：

|  41

38

|  31

插入 12 后，红黑树为：

|  41

38

|  31

|  |  12

插入 19 后，红黑树为：

|  41

38

|  |  31

|  19

|  |  12

插入 8 后，红黑树为：

|  41

38

|  |  31

|  19

|  |  12

|  |  |  8



## 13.4 删除

- $y$ 为从树中删除的结点或者指被移动至删除结点位置处的节点
- 为了尽可能的保持红黑树的性质，需要将 $y$ 的颜色设置为 $z$ 的颜色
- 如果移动或删除的 $y$ 是黑色结点，为了维持性质4， $y$ 移动后，在之前 $y$ 位置处的结点 $x$ 会加上一层额外的黑色，
即双重黑色，或者红黑色，需要针对此种情况对红黑树的性质进行修复

#### $\mathrm{RB-DELETE-FIXUP}$

- 进入 $\mathrm{RB-DELETE-FIXUP}$ 的前提是 $y$ 为黑色
- 如果 $x$ 为红色，则真接将其置为黑色即可
- 如果 $x$ 为黑色，则分为四种情况
  - $x$ 是其父亲的左子结点和右子结点，是两种基本对称的情况，所以只研究 $x$ 是其父新的左子结点即可

##### 情况1： $x$ 的兄弟结点 $w$ 是红色的

- 通过颜色变换和右旋操作，将其转换为情况2 ~ 情况4，即 $w$ 是黑色的
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216173336.png width=600>

##### 情况2： $w$ 是黑色的，且 $w$ 的两个子结点也是黑色的

- 此种情况，可以将 $x.parent$ 加上一层额外的黑色，同时将 $w$ 置为红色
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216173425.png width=600> 

##### 情况3：$w$ 是黑色的，且 $w$ 的左孩子是红色，右孩子是黑色

- 通右旋操作，并更新 $w$ 结点，将 $w$ 的右孩子变为红色，转换为情况4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216173728.png width=600>

##### 情况4： $w$ 是黑色的，且 $w$ 的右孩子是红色的

- 将 $x.parent$ 改为黑色，并对 $x.parent$ 执行左旋, 此时即可以去掉 $x$ 的双重黑色
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200216174226.png width=600>


#### 代码实现

In [0]:
def transplant(self, u, v):
  """在红黑树是移动子树"""
  if u.parent == RBTree.NIL:
    self.root = v
  elif u.parent.left == u:
    u.parent.left = v
  else:
    u.parent.right = v
  v.parent = u.parent

RBTree.transplant = transplant

In [0]:
def delete(self, z):
  """删除某个结点，同时保证红黑树的性质"""
  y = z
  y_original_color = y.color
  if z.left == RBTree.NIL:
    x = z.right
    self.transplant(z, z.right)
  elif z.right == RBTree.NIL:
    x = z.left
    self.transplant(z, z.left)
  else:
    y = self.minimum(z.right)
    y_original_color = y.color
    x = y.right
    if y.parent == z:
      x.parent = y  # 此时 x 可能为 NIL，所以需要对其 parent 特殊赋值，其它的赋值均在 transplant 中完成
    else:
      self.transplant(y, y.right)
      y.right = z.right
      y.right.parent = y
    self.transplant(z, y)
    y.left = z.left
    y.left.parent = y
    y.color = z.color
  if y_original_color == RBNode.B:
    self.delete_fixup(x)

RBTree.delete = delete

In [0]:
def delete_fixup(self, x):
  while x != self.root and x.color == RBNode.B:
    if x == x.parent.left:
      w = x.parent.right
      if w.color == RBNode.R:  # case 1
        x.parent.color = RBNode.R
        w.color = RBNode.B
        self.left_rotate(x.parent)
        w = x.parent.right  # 让下面的 case 能正常执行
      if w.left.color == RBNode.B and w.right.color == RBNode.B:  # case 2
        w.color = RBNode.R
        x = x.parent
      else:
        if w.right.color == RBNode.B:  # case 3
          w.left.color = RBNode.B
          w.color = RBNode.R
          self.right_rotate(w)
          w = x.parent.right
        # case 4
        w.color = x.parent.color
        x.parent.color = RBNode.B
        w.right.color = RBNode.B
        self.left_rotate(x.parent)
        x = self.root # 让下一次循环退出循环，同时确保根为黑色
    else:
      w = x.parent.left
      if w.color == RBNode.R:  # case 1
        x.parent.color = RBNode.R
        w.color = RBNode.B
        self.right_rotate(x.parent)
        w = x.parent.left  # 让下面的 case 能正常执行
      if w.left.color == RBNode.B and w.right.color == RBNode.B:  # case 2
        w.color = RBNode.R
        x = x.parent
      else:
        if w.left.color == RBNode.B:  # case 3
          w.right.color = RBNode.B
          w.color = RBNode.R
          self.left_rotate(w)
          w = x.parent.left
        # case 4
        w.color = x.parent.color
        x.parent.color = RBNode.B
        w.left.color = RBNode.B
        self.right_rotate(x.parent)
        x = self.root # 让下一次循环退出循环，同时确保根为黑色
      

  x.color = RBNode.B #针对红黑色的情况


RBTree.delete_fixup = delete_fixup

In [0]:
rbt = RBTree()
for key in [41, 38, 31, 12, 19, 8]:
  rbt.insert(RBNode(key=key))

print("原始的红黑树为：\n{}".format(rbt))

for key in [8, 12, 19, 31, 38, 41]:
  rbt.delete(rbt.search(rbt.root, key))
  print("删除 {} 后，红黑树为：\n{}".format(key, rbt))

原始的红黑树为：

|  41

38

|  |  31

|  19

|  |  12

|  |  |  8

删除 8 后，红黑树为：

|  41

38

|  |  31

|  19

|  |  12

删除 12 后，红黑树为：

|  41

38

|  |  31

|  19

删除 19 后，红黑树为：

|  41

38

|  31

删除 31 后，红黑树为：

|  41

38

删除 38 后，红黑树为：

41

删除 41 后，红黑树为：
None


### 练习

#### 13.4-1
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225090726.png width=800> 

- 退出循环有两种情况
  1. $x$ 为根，此时在代码的最后会将 $x$ 的颜色设为黑色
  2. $x$ 的颜色为红色，在代码的最后会将 $x$ 的颜色设为黑色，此时不管$x$ 是否为根，均不影响根结点是黑色的
- 循环中的 4 种情况，执行结束后，没有修改最顶层结点的颜色，也就不会影响根结点的颜色

###### 循环四种情况的示意图

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/1.png width=800>

#### 13.4-2
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225091654.png width=800>

- 函数末尾的语句 $x$ 的颜色置为黑色，即可在 $x$ 和 $x.p$ 均为红色的情况下恢复性质 4

#### 13.4-3
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225092524.png width=800>

- 原始红黑树
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225092558.png width=600>
- 删除 8
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225093005.png width=600>
- 删除 12
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225093355.png width=500>
- 删除 19
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225125011.png width=400>
- 删除 31
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225125605.png width=200>
- 删除 38
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225135642.png width=50>
- 删除 41
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225130119.png width=50>



#### 13.4-4
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225133859.png width=800>

- 如果 $y$ 没有孩子结点，则 $x$ 为 $T.nil$，如此在程序刚开始运行时，在第1，2 行会对 $T.nil$ 进行检查，
且在第一次执行循环的过程中，涉及到对 $x$ 除赋值外操作，均是对 $T.nil$ 进行操作
  - 此时如果 $y$ 为根结点(即删除的结点为红黑树中仅存的结点)，则第 23 行还会将 $T.nil$ 的颜色置为
  黑色
- 进入 $while$ 循环后， $x$ 为双重黑色，所以其兄弟结点 $w$ 不可能为 $T.nil$，因此涉及到 $w$ 的部分不会
对 $T.nil$ 进行操作


###### RB-DELETE-FIXUP 的伪代码

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225141952.png width=800>

#### 13.4-5
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225145219.png width=800>

- 计数包括根结点，同时计算额外的黑色
  - Case1: 变换前后 $x$ 均为双重黑色
    - $\alpha, \beta$ 在变换前后的结果为 $3$
    - $\gamma, \delta, \varepsilon, \xi$ 在变换前后的结果均为 $2$
  - Case2: 变换前 $x$ 为双重黑色， 变换后 $c$ 额外添加了一层黑色
    - $\alpha, \beta$ 子树在变换前后的结果均为 $2 + count(c)$
    - $\gamma, \delta, \varepsilon, \xi$ 在变换前后的结果均为 $2 + count(c)$
  - Case3: 变换前后 $x$ 均为双重黑色
    - $\alpha, \beta$ 子树在变换前后的结果均为 $2 + count(c)$
    - $\gamma, \delta$ 在变换前后的结果均为 $1 + count(c)$
    - $\varepsilon, \xi$ 在变换前后的结果均为 $2 + count(c)$
  - Case4：变换前 $x$ 为双重黑色结点，变换后没有包含额外黑色的结点
    - $\alpha, \beta$ 子树在变换前后的结果均为 $2 + count(c)$
    - $\gamma, \delta$ 在变换前后的结果均为 $1 + count(c) + count(c^{'})$
    - $\varepsilon, \xi$ 在变换前后的结果均为 $1 + count(c)$

###### 循环四种情况的示意图

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/1.png width=800>

#### 13.4-6
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225153215.png width=800>

- 如果 $x$ 的兄弟结点 $w$ 是红色的，则由性质4 可知， $x$ 和 $w$ 共有的父结点 $x.p$ 一定是红色的

#### 13.4-7

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225161856.png width=800>

- 不一样，例子如下：
  - 初始的红黑树
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225162031.png width=200>
  - 插入 1
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225162141.png width=250>
  - 删除 1
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225162230.png width=200>


## 思考题

### 13-1 持久动态集合
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200225170559.png width=800>

#### a

- 插入结点
  - 设插入的结点为 $z$，则需要修改从根结点到结点$z$ 的简单路径上的所有结点
- 删除结点
  - 需要分为两种情况
    - 如果 $y$ 只有一个子结点或者没有子结点，则需要修改从根x到结点 $y$ 的简单路径上的所有结点
    - 如果 $y$ 有两个子结点，设 $z$ 为 $y$ 的后继，则需要修改从根结点到结点 $z$ 的简单路径上的
    所有结点

#### b

In [0]:
class TreeNodeWithoutParent():
  def __init__(self, key, left=None, right=None):
    self.key = key
    self.left = left
    self.right = right

  def __repr__(self):
    return str(self.key)

In [0]:
def persistent_tree_insert(root, z):
  """返回插入结点 z 后的新树"""
  if root is None:
    return z
  new_node = TreeNodeWithoutParent(root.key, root.left, root.right)
  if z.key < root.key:
    new_node.left = persistent_tree_insert(root.left, z)
  else:
    new_node.right = persistent_tree_insert(root.right, z)
  return new_node

In [0]:
def print_tree(root):
  """借助二叉搜索树逆时针旋转90度打印一颗树"""
  print(BinarySearchTree(root))

In [0]:
history = [None]
root = None
for i in [4, 2, 5, 1, 6]:
  root = persistent_tree_insert(root, TreeNodeWithoutParent(i))
  history.append(root)
for root in history:
  print_tree(root)
  print("- "*5)

None
- - - - - 

4

- - - - - 

4

|  2

- - - - - 

|  5

4

|  2

- - - - - 

|  5

4

|  2

|  |  1

- - - - - 

|  |  6

|  5

4

|  2

|  |  1

- - - - - 


#### c

- 设树高为 $h$， 则 PERSISTENT-TREE-INSERT 的时间和空间复杂度均为 $O(lg(h))$

#### d

- 因为每次插入新节点时，必会更新根节点，如果增加父节点属性，则必须要更新根节点的子结点。以此往下递推，则树中的每个结点都需要被
更新，因此时间和空间需求均为 $\Omega(n)$

#### e

- 红黑树可保证树高为 $O(\lg n)$，其插入和删除只是比二叉搜索树多了几次旋转操作，因为旋转所导致的需要更新的结点数也只是常数个，因
此可保证每次插入和删除的最坏情况运行时间和空间均为 $O(lg(n))$

### 13-2 红黑树上的连接操作
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304215327.png width=800>

#### a

- RB-INSERT 插入的是红色节点，在维护红黑树的性质时，只有在情况 1 将根节点置为红色时，会导致黑高加 1， 其它情况不会影响黑高，
因此维护黑高不会影响渐近运行时间。或者插入的结点本身即为根节点，此时也会导致黑高加 1
- RB-DELETE 在删除元素时，会引入额外的黑色，不会影响整棵树的黑高
 - 在维护红黑树性质的过程中，除了情况 2 可能会将额外的黑色移至根节点，导致黑高减 1 ； 其余的情况不会影响整棵树的黑高，因此维护黑高也不会影响渐近运行时间。
 - 当树由一个结点变为空树时，也需要将黑高减 1
- 沿 $T$ 下降时，遇到黑色结点将黑高减 1， 红色节点黑高不变，因此对每个被访问的节点，可在 $O(1)$ 的时间内，确定其黑高

#### b

- 在 $T_1$ 中沿着右脊柱往下遍历，直至找到黑高为 $T_2.bh$ 的黑色结点

#### c

- 用结点 $x$ 取代结点 $y$， 然后将 $T_y$ 作为 $x$ 的左子树， $T_2$ 作为 $x$ 的右子树

#### d

- 为了保证性质 1， 3， 5，需要将 $x$ 变为红色
- 为了维护性质 2 和 4， 可以直接调用 RB-INSERT-FIXUP 来实现，时间代价为 $O(\lg n)$

#### e

- $T_1.bh \le T_2.bh$ 时，在 $T_2$ 中从黑高为 $T_1.bh$ 的结点中选出具有最小关键字的黑色结点 $y$
- 将 $x$ 着为红色， 用 $x$ 替代 $y$ 的位置，然后将 $T_1$ 作为 $x$ 的左子树， $T_y$ 作为 $x$ 的右子树
- 借助 RB-INSERT-FIXUP 来修复 $x$ 为红色对红黑树所带来的破坏

#### f

- 查找节点 $y$ 需用时 $O(\lg n)$
- 合并子树，即 $c$ 中的步骤用时为 $O(1)$
- 维护红黑树的性质，需要用时 $O(\lg n)$
- 因此， RB-JOIN 的运行时间是 $O(\lg n)$

### 13-3 AVL 树
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304215440.png width=800>

#### a

- 设高度为 $h$ 的树有 $T_h$ 个结点， 则由 AVL 树的性质可得： $$T_{h} \le T_{h-1} + T_{h-2}$$
- 可由归纳法证明 $$\begin{aligned}
n = T_h \ge F_h = \lfloor {\Phi^h \over \sqrt{5}} + {1 \over 2} \rfloor > {\Phi^h \over \sqrt{5}} 
\end{aligned}$$
- 由上式可得： $$h < lg_{\Phi} \sqrt{5}n = O(lg(n))$$

#### b

###### 分析

- 假设 $x$ 的右侧子树的高度大于左侧子树，则会存在两种情况
  - $x.right.right.h \ge x.right.left.h$ 
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304191130.png width=500>
  - $x.right.right.h \le x.right.left.h$ 
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304191215.png width=500>

###### 代码实现

- 添加一个哨兵节点作为叶子结点，其高度设为 $-1$

In [0]:
class AVLTreeNode(BinaryTreeNode):
  def __init__(self, key=None, left=None, right=None, parent=None, h=0):
    super().__init__(key=key, left=left, right=right, parent=parent)
    self.h = h
  
  def __eq__(self, other):
    return self.key == None if other is None else self is other  # 为了兼容二叉搜索树中的相关函数

In [0]:
class AVLTree(RBTree):
  NIL = AVLTreeNode(h=-1)
  
  def __init__(self, root=NIL):
    super().__init__(root)
  
  def left_rotate(self, x):
    """旋转后需要更新相应结点的高度"""
    super().left_rotate(x)   
    x.h = max(x.left.h, x.right.h) + 1
    x.parent.h = max(x.parent.left.h, x.parent.right.h) + 1
    return x.parent

  def right_rotate(self, x):
    """旋转后需要更新相应结点的高度"""
    super().left_rotate(x)   
    super().right_rotate(x)   
    x.h = max(x.left.h, x.right.h) + 1
    x.parent.h = max(x.parent.left.h, x.parent.right.h) + 1
    return x.parent

  def balance(self, x):
    if abs(x.left.h - x.right.h) <= 1:
      return x
    if x.left.h > x.right.h:
      if x.left.left.h < x.left.right.h:
        self.left_rotate(x.left)
      return self.right_rotate(x)
    else:
      if x.right.right.h < x.right.left.h:
        self.right_rotate(x.right)
      return self.left_rotate(x)
  
  def delete(self, z):
    """"""
    pass
    

#### c

##### 分析

- 使用递归来完成插入和节点高度性质的维护

##### 代码实现

In [0]:
def insert(self, z):
  def helper(root, z):
    if root == AVLTree.NIL:
      z.left = AVLTree.NIL
      z.right = AVLTree.NIL
      z.parent = AVLTree.NIL
      return z
    if z.key < root.key:
      y = helper(root.left, z)
      root.left = y
      y.parent = root
      root.h = y.h + 1
    else:
      y = helper(root.right, z)
      root.right = y
      y.parent = root
      root.h = y.h + 1
    return self.balance(root)
  self.root = helper(self.root, z)

AVLTree.insert = insert


In [0]:
avl = AVLTree()

In [0]:
for i in range(10):
  avl.insert(AVLTreeNode(i))

In [0]:
print(avl)


|  |  |  9

|  |  8

|  7

|  |  |  6

|  |  5

|  |  |  4

3

|  |  2

|  1

|  |  0



#### d

- 插入和更新节点高度的过程用时为 $O(h)$，即 $O(lg(n))$
- 如果 $x$ 为不平衡树， 则 `balance(x)`一旦执行 ，则会导致原子树的树高减1，因此在后续的递归过程中，
$x$ 却不可能为不平衡树，因此旋转最多执行 2 次，即 $O(1)$ 次旋转



### 13.4 treap 树
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304220006.png width=800>

#### a
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304220050.png width=800>

- 可以从 treap 树的性质上进行证明
- 为了满足最小堆的性质，优先级最小的节点一定为根节点。同时关键字比根节点小的节点，一定位于左子树中，其余
元素一定位于右子树中。对左，右子树进行递归，便可以得到唯一的 treap 树

#### b
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304220201.png width=800>

- 随机构造的二叉搜索树的高度为 $O(\lg n)$
- 结点添加优先级后，等价于优先级高的结点先插入，优化级低的结点后插入，由于各个结点的优先级是随机生成的，所以等价于以随机的顺序插入各个结点，即等价于随机构造的二叉搜索树，因此其高度为 $O(\lg n)$
- $n$ 个结点的二叉搜索树高度至少为 $\Omega(\lg n)$
- 综上， Treap 树的高度为 $\Theta(\lg n)$


#### c
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304220430.png width=800>

In [0]:
class TreapTreeNode(BinaryTreeNode):
  def __init__(self, key=None, left=None, right=None, parent=None, priority=None):
    super().__init__(key=key, left=left, right=right, parent=parent)
    self.priority = 1/random.random() if priority is None else priority

In [0]:
class TreapTree(RBTree):
  def __init__(self, root=None):
    self.root = root
  
  def insert(self, z):
    """向 treap 树中插入一个元素"""
    x = self.root
    y = None
    while x != None:
      y = x
      if z.key < x.key:
        x = x.left
      else:
        x = x.right
    z.parent = y
    if y == None:
      self.root = z
    elif z.key < y.key:
      y.left = z
    else:
      y.right = z
    
    # 维护 treap 树的性质
    while y != None:
      if y.priority < z.priority:
        return
      if y.left == z:
        self.right_rotate(y)
      else:
        self.left_rotate(y)
      y = z.parent

  def print_priority(self):
    def helper(x, i):
      res = ''
      if x != None:
        res += helper(x.right, i+1)
        res += "\n" + "|    " * i + "{:.2f}\n".format(x.priority)
        res += helper(x.left, i+1)
      return res

    if self.root == None:
      print("None")
    else:
      print(helper(self.root, 0))


In [0]:
tt = TreapTree()
for i in range(10):
  tt.insert(TreapTreeNode(i))
print("关键字组成的树为：\n{}".format(tt))
print("- "*10)
print("优先级组成的最小堆为:")
tt.print_priority()

关键字组成的树为：

|  |  |  9

|  |  8

|  |  |  7

|  6

|  |  5

|  |  |  4

3

|  |  2

|  |  |  1

|  0

- - - - - - - - - - 
优先级组成的最小堆为:

|    |    |    2.63

|    |    1.90

|    |    |    3.50

|    1.46

|    |    1.60

|    |    |    34.47

1.08

|    |    8.35

|    |    |    150.96

|    1.19



#### d
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200305084058.png width=500>

- TREAM-INSERT 在执行的过程中，只会沿着简单路径进行查找和旋转操作，而树高为 $\Theta(\lg n)$，因此期望的
运行时间也为 $\Theta(\lg n)$

#### e
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200305090016.png width=800>

- 证明可借助循环不变式来实现
- 当结点 $x$ 刚插入 $T$，还没有进行旋转操作时， $C_0 + D_0 = 0$，满足条件
- 假设当进行 $k$ 次旋转操作后，进行的旋转次数为 $C_k + D_k$
  - 如果 $k+1$ 次为 $x$ 父结点的左旋操作， 则 $x$ 的右子树不变，即 $D_{k+1}=D_k$，而 $x$ 左子树的右脊柱加 $1$，即
  $C_{k+1}=C_{k}+1$，由此可得 $C_{k+1} + D_{k+1}$ 与总的旋转次数相同
  - 如果 $k+1$ 次旋转为 $x$ 父结点的右旋操作，由对称性可得同样的结论
- 综上可得，在插入 $x$ 期间执行的旋转的总次数等于 $C+D$

#### f
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200305090342.png width=800>

- 证明 $\rightarrow$
  - $X_{ik}=1$ 说明 $y$ 在 $x$ 的左子树的右脊柱中
    - 由二叉搜索树的性质，可得 $y.key<x.key$
    - 由最小堆的性质可得 $y.priority > x.priority$
    - 如果 $y.key<z.key<x.key$，则由中序遍历的顺序可知， $z$ 位于 $y$ 的右子树中， 即可得 $y.priority<z.priority$
- 证明 $\leftarrow$
  - 采用反证法
    1. 如果 $x, y$ 没有父子关系，则可取 $z$ 为它们共同的祖先，如此 $y.priority > z.prioriyt$，不满足条件 3
    2. 设 $y$ 是 $x$ 的父辈结点，则不符合条件 1
    3. 由 1， 2 可得 $x$ 是 $y$ 的父辈节点，且由条件 2 可得 $y$ 只能位于 $x$ 的左子树中
    4. 设 $y$ 不位于 $x$ 左子树的右脊柱中，则取 $z$ 为 $x$ 左子树右脊柱上的点，且 $y$ 位于 $z$ 的左子树中，即可
    满足 $y.key<z.key<x.key$，但是 $y.priority > z.priority$
    5. 结合 1，2，3，4 可得 $y$ 在 $x$ 的左子树的右脊柱中，即 $X_{ik}=1$

#### g
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304221051.png width=600>


- 题中的概率需要假定 $i < k$
- 证明：
  - $X_{ik} = 1$ 需要满足 $h$ 中的三个条件，将 $i, k$ 中的数按照优先级进行排列，则共有 $(k-i+1)!$ 中可能的排列
  1. 为了满足条件1，则 $k$ 必须排在 $i$ 之前
  2. 为了满足条件3，则对于任意的 $x \in (i, k)$， $x$ 需要排在 $i$ 之后
  - 由 1， 2 可得， $k$ 的优化级最小， $i$ 的优化级次之，剩余的元素共有 $(k-i-1)!$ 中可能排列，所以可得：
  $$\Pr\{X_{ik} = 1 \} = {(k-i-1)! \over (k-i+1)!} = {1 \over (k-i+1)(k-i)}$$

#### h
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304221121.png width=600>


1. 如果 $j > k$, 则 $\Pr\{X_{ik}=1\} = 0$
- 由 1 结合 $g$ 中的分析可得：
$$
\begin{aligned}
E[C] &=E\left[\sum_{i=1}^{k-1} X_{i, k}\right] \\
&=\sum_{i=1}^{k-1} E\left[X_{i, k}\right] \\
&=\sum_{i=1}^{k-1} \operatorname{Pr}\left[X_{i, k}=1\right] \\
&=\sum_{i=1}^{k-1} \frac{1}{(k-i+1)(k-i)}  && 令 j = k - i\\ 
&=\sum_{j=1}^{k-1} \frac{1}{(j+1)(j)} \\
&=\sum_{j=1}^{k-1}\left(\frac{1}{j}-\frac{1}{j+1}\right) \\
&=\frac{1}{1}-\frac{1}{k} \\
&=1-\frac{1}{k}
\end{aligned}
$$

#### i
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304221138.png width=600>


- 当 $x.key = k$ 时，左子树中共有 $k - 1$ 个元素， 右子树中共有 $n-k$ 个元素，由对称性即可得：
  $$E[D] = 1 - {1 \over n - k + 1}$$

#### j
- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200304221204.png width=800>

- $$E[C] + E[D] = 2 - {1 \over k} - {1 \over n - k + 1} \le 2$$
- 由此可得执行旋转的期望次数小于 2